In [10]:
import pickle
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.stats import pearsonr
from sklearn.metrics import mutual_info_score
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data



# Underlying SCM

In [11]:
# links_coeffs = {
#     0: [((0, -1), 0.7, lin_f), ((1, -1), -0.8, lin_f)],
#     1: [((1, -1), 0.8, lin_f), ((3, -1), 0.8, lin_f)],
#     2: [((2, -1), 0.5, lin_f), ((1, -2), 0.5, lin_f), ((3, -3), 0.6, lin_f)],
#     3: [((3, -1), 0.4, lin_f)]
# }

# Loading data

In [14]:
df = pd.read_pickle('tig_df.pkl')

# GNN

In [15]:
class TimeSeriesGNN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(TimeSeriesGNN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

def prepare_data(df, max_lag=3):
    # Assume df is your pandas DataFrame
    features = []
    edges = []
    target = []

    for t in range(max_lag, len(df)):
        # Node features
        features.append(torch.tensor([
            df.iloc[t, df.columns.get_loc('x0')],
            df.iloc[t-1, df.columns.get_loc('x1')],
            df.iloc[t-1, df.columns.get_loc('x3')],
            df.iloc[t-2, df.columns.get_loc('x1')],
            df.iloc[t-3, df.columns.get_loc('x3')],
        ], dtype=torch.float))

        # Edges (now we're creating edges for each time step)
        edges.extend([
            [t-max_lag, t-max_lag+1],  # x1(t-1) -> x0(t)
            [t-max_lag+1, t-max_lag+2],  # x3(t-1) -> x1(t)
            [t-max_lag+3, t-max_lag+2],  # x1(t-2) -> x2(t)
            [t-max_lag+4, t-max_lag+2],  # x3(t-3) -> x2(t)
        ])

        # Target
        target.append(df.iloc[t, df.columns.get_loc('x0')])

    x = torch.stack(features)
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    y = torch.tensor(target, dtype=torch.float)

    # Print shapes for debugging
    print(f"x shape: {x.shape}")
    print(f"edge_index shape: {edge_index.shape}")
    print(f"y shape: {y.shape}")

    return Data(x=x, edge_index=edge_index, y=y)

# Prepare the data
data = prepare_data(df)

# Initialize the model
model = TimeSeriesGNN(num_features=5, hidden_channels=16, num_classes=1)

# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out.squeeze(), data.y)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(data.x, data.edge_index)

print(f"Predictions shape: {predictions.shape}")

x shape: torch.Size([997, 5])
edge_index shape: torch.Size([2, 3988])
y shape: torch.Size([997])


RuntimeError: index 997 is out of bounds for dimension 0 with size 997